# ETL

## Extracción

In [18]:
import pandas as pd

In [19]:
file_path = '../data/raw/base_datos_colombia.xlsx'
xls = pd.read_excel(file_path, sheet_name=None, engine='openpyxl')

In [20]:
# Convertir las hojas del archivo Excel en DataFrames
df_compras = xls['compras']
df_ventas = xls['ventas']
df_productos = xls['productos']
df_proveedores = xls['proveedores']
df_promociones = xls.get('promociones')

## Transformación

### Typecasting

In [21]:
# Typecasting para df_compras
df_compras = df_compras.astype({
    'fecha': 'datetime64[ns]',
    'sucursal': 'category',
    'producto_id': 'category',
    'unidades_compradas': 'int',
    'costo_unitario': 'float',
    'proveedor_id': 'category'
})

In [22]:
df_ventas = df_ventas.astype({
    'fecha': 'datetime64[ns]',
    'sucursal': 'category',
    'producto_id': 'category',
    'unidades_vendidas': 'int',
    'precio_venta': 'float'
})

In [23]:
df_productos = df_productos.astype({
    'producto_id': 'category',
    'nombre_producto': 'category',
    'categoria': 'category',
    'marca': 'category',
    'unidad_medida': 'category'
})

In [24]:
df_proveedores = df_proveedores.astype({
    'proveedor_id': 'category',
    'nombre_proveedor': 'category',
    'calificacion_proveedor': 'float',
    'plazo_entrega_dias': 'int'
})

In [25]:

if df_promociones is not None:
    df_promociones = df_promociones.astype({
        'producto_id': 'category',
        'fecha_inicio': 'datetime64[ns]',
        'fecha_fin': 'datetime64[ns]',
        'tipo_promocion': 'category',
        'descuento_porcentual': 'float'
    })

### Consolidación de tabla única

#### 1. Crear una base de combinación de fechas, sucursales y productos

In [26]:
# Rango de fechas
fecha_min = min(df_compras['fecha'].min(), df_ventas['fecha'].min())
fecha_max = max(df_compras['fecha'].max(), df_ventas['fecha'].max())
fechas = pd.date_range(fecha_min, fecha_max, freq='D')

# Todas las combinaciones posibles
base = pd.MultiIndex.from_product(
    [fechas, df_ventas['sucursal'].unique(), df_ventas['producto_id'].unique()],
    names=['fecha', 'sucursal', 'producto_id']
).to_frame(index=False)


#### 2. Integrar `df_ventas` y `df_compras` compras a la base

In [27]:
# Agrupar ventas por fecha, sucursal y producto
ventas_agg = (
    df_ventas
    .groupby(['fecha', 'sucursal', 'producto_id'], observed=True)
    .agg({
        'unidades_vendidas': 'sum',
        'precio_venta': 'mean' # Por si se vende un producto a diferentes precios
    })
    .reset_index()
)


In [28]:
# Agrupar compras por fecha, sucursal y producto
compras_agg = (
    df_compras
    .groupby(['fecha', 'sucursal', 'producto_id'], observed=True)
    .agg({
        'unidades_compradas': 'sum',
        'costo_unitario': 'mean' # Por si se compra un producto a diferentes precios
    })
    .reset_index()
)


In [29]:

# Merge con la base
df = base.merge(ventas_agg, on=['fecha', 'sucursal', 'producto_id'], how='left')
df = df.merge(compras_agg, on=['fecha', 'sucursal', 'producto_id'], how='left')


In [30]:
# Rellenar valores nulos con 0
df['unidades_vendidas'] = df['unidades_vendidas'].fillna(0).astype(int)
df['precio_venta'] = df['precio_venta'].fillna(0)

df['unidades_compradas'] = df['unidades_compradas'].fillna(0).astype(int)
df['costo_unitario'] = df['costo_unitario'].fillna(0)


#### 3. Agregar datos de productos

In [31]:
df = df.merge(df_productos, on='producto_id', how='left')


#### 4. Calcular el márgen bruto

In [32]:
df['margen_bruto'] = df['precio_venta'] - df['costo_unitario']


#### 5. Flag de promociones (identificar si hay o no promoción ese día)

In [33]:
if df_promociones is not None:
    # Expande las promociones a nivel día-producto
    promos_expandidas = []
    for _, row in df_promociones.iterrows():
        dias = pd.date_range(row['fecha_inicio'], row['fecha_fin'])
        for fecha in dias:
            promos_expandidas.append({
                'producto_id': row['producto_id'],
                'fecha': fecha,
                'tipo_promocion': row['tipo_promocion'],
                'descuento_porcentual': row['descuento_porcentual']
            })

    df_promos_exp = pd.DataFrame(promos_expandidas)

    # Merge con la tabla principal
    df = df.merge(df_promos_exp, on=['producto_id', 'fecha'], how='left')

    # Crea flags
    df['producto_en_promocion'] = df['descuento_porcentual'].notna().astype(int)
else:
    df['producto_en_promocion'] = 0


## Carga

In [34]:
df.to_csv('../data/processed/tabla_consolidada.csv', index=False)